# Uncovering real estate investment opportunities in Madrid 

The objective of this project is to understand the general business of Airbnb in Madrid. In order to use it as a proxy to identify the best neighbourhoods to invest in real state. 

Thanks to http://insideairbnb.com/get-the-data.html we have got a dataset with information from Airbnb updated on 14 May 2019.

In [88]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sns
airbnb = pd.read_csv("data.csv")
airbnb.head(3)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,6369,"Rooftop terrace room with ensuite bathroom, Airc.",13660,Simon,Chamartín,Hispanoamérica,40.45628,-3.67763,Private room,70,3,63,2019-04-30,0.56,2,73
1,21853,Bright and airy room,83531,Abdel,Latina,Cármenes,40.40341,-3.74084,Private room,17,4,33,2018-07-15,0.59,2,0
2,24805,Gran Via Studio Madrid,101471,Iraido,Centro,Universidad,40.42202,-3.70395,Entire home/apt,60,5,2,2017-07-03,0.04,1,355


In [89]:
airbnb = pd.read_csv("data.csv")
print(airbnb.shape)
airbnb.describe()

(18808, 16)


,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
count,1.880800e+04,1.880800e+04,18808.000000,18808.000000,18808.00000,18808.000000,18808.000000,14860.000000,18808.000000,18808.000000
mean,2.097478e+07,8.627014e+07,40.420267,-3.695638,122.31678,4.092726,34.853732,2.054841,12.150468,150.284666
std,9.810985e+06,7.749516e+07,0.020824,0.026134,360.60064,18.280586,59.023326,1.966534,30.255443,128.518601
min,6.369000e+03,5.154000e+03,40.319600,-3.886430,1.00000,1.000000,0.000000,0.010000,1.000000,0.000000
25%,1.424060e+07,1.872099e+07,40.409867,-3.707570,37.00000,1.000000,1.000000,0.480000,1.000000,23.000000
50%,2.190276e+07,6.103789e+07,40.418510,-3.701520,60.00000,2.000000,9.000000,1.425000,2.000000,124.000000
75%,2.953546e+07,1.388107e+08,40.427620,-3.692240,98.00000,3.000000,42.000000,3.110000,8.000000,286.000000
max,3.469610e+07,2.617170e+08,40.580150,-3.520260,9999.00000,1124.000000,573.000000,20.530000,230.000000,365.000000


For this analysis, we are only going to use those ads that rent the whole apartment
It would be great to have the number of rooms per apartment to reach a more granular level. However, this information is not included in the dataset 

In [90]:
#removing price outliers

airbnb_clean = airbnb[airbnb["price"]<250]
airbnb_clean = airbnb_clean[airbnb_clean["price"]>15]

#for this analisys we are only going to use  those adds that rent the whole aparment
airbnb_clean = airbnb_clean[airbnb_clean["room_type"] == "Entire home/apt"]

airbnb_clean.shape


(10956, 16)

In [91]:
#checking the number of registers per neighbourhood 
#we are going to define a minimum threshold of 50 apartments for neighbourhood

neighbourhood = airbnb_clean["neighbourhood"].value_counts()
print("There were", neighbourhood.count(), "neighbourhoods in the dataset")
neighbourhood = neighbourhood[neighbourhood > 50]
print("After removing those with just a few apartments, we have", neighbourhood.count())
neighbourhood_list = neighbourhood.index

There were 123 neighbourhoods in the dataset
After removing those with just a few apartments, we have 38


In [92]:
#filtering out the neighbourhoods with just a few apartments

neighbourhood_list = list(neighbourhood_list)
bool_neighbourhood = list(map(lambda x: x in neighbourhood_list, airbnb_clean["neighbourhood"]))
airbnb_clean = airbnb_clean[bool_neighbourhood]

In [93]:
#now let´s create a table with the information we are interested in:
    # number of apartments per neighbourhood
    # median latitude and longitude for the map representation
    # average price of the apartment

table_neighbourhood = airbnb_clean.pivot_table( 
                                  values=['id','latitude', 'longitude', 'price'], 
                                  index=['neighbourhood'], 
                                  aggfunc={'id': pd.Series.nunique, 'latitude': np.mean, 'longitude': np.mean, 'price': np.mean})

table_neighbourhood.head(3)

,id,latitude,longitude,price
neighbourhood,,,,
Acacias,90,40.402446,-3.705229,74.011111
Adelfas,59,40.400430,-3.670911,72.050847
Almagro,114,40.433821,-3.694432,100.254386


In [94]:
table_neighbourhood.to_csv(r'table_neighbourhood.csv')

In the following map, the data from the previous table is represented. 

The main goal of this project is therefore achieved. Since in the map, it is possible to recognize areas of Madrid with high demand (high prices) and not a very high offer (low number of apartments)

In [97]:
%%HTML
<div class='tableauPlaceholder' id='viz1561470500183' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Un&#47;UncoveringrealestateinvestmentopportunitiesinMadrid&#47;Dashboard1&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='path' value='views&#47;UncoveringrealestateinvestmentopportunitiesinMadrid&#47;Dashboard1?:embed=y&amp;:display_count=y&amp;publish=yes' /> <param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Un&#47;UncoveringrealestateinvestmentopportunitiesinMadrid&#47;Dashboard1&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1561470500183');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

If you want to play further with this map you can also find it at: https://public.tableau.com/views/UncoveringrealestateinvestmentopportunitiesinMadrid/Dashboard1?:embed=y&:display_count=yes&publish=yes&:origin=viz_share_link